In [1]:
# Import dependencies.
import pandas as pd
import numpy as np
import os
import dask.dataframe as dd
from tqdm.auto import tqdm
pd.set_option('display.max_columns', 500)

In [2]:
# Set path to files and read in the CSV file. Use dtyping to save memory and use dask for efficiency.
path = 'C:/Users/outla/Desktop/VA_ML_Research/UCSD_NRD_2017'
os.chdir(path)
cohort_file = dd.read_csv("Nghia_NRD_2017_Cohort.csv", dtype={'age':'float16', 'aweekend':'float16', 'died':'float16', 'discwt':'float64', 'dispuniform':'float16', 'dmonth':'float16', 'dqtr':'float16', 'drg':'float16', 'drgver':'float16', 'drg_nopoa':'float16', 'i10_dx1':str, 'i10_dx2':str, 'i10_dx3':str, 'i10_dx4':str, 'i10_dx5':str, 'i10_dx6':str, 'i10_dx7':str, 'i10_dx8':str, 'i10_dx9':str, 'i10_dx10':str, 'i10_dx11':str, 'i10_dx12':str, 'i10_dx13':str, 'i10_dx14':str, 'i10_dx15':str, 'i10_dx16':str, 'i10_dx17':str, 'i10_dx18':str, 'i10_dx19':str, 'i10_dx20':str, 'i10_dx21':str, 'i10_dx22':str,'i10_dx23':str,'i10_dx24':str, 'i10_dx25':str,'i10_dx26':str,'i10_dx27':str,'i10_dx28':str,'i10_dx29':str, 'i10_dx30':str, 'i10_dx31':str, 'i10_dx32':str, 'i10_dx33':str, 'i10_dx34':str, 'i10_dx35':str, 'i10_dx36':str, 'i10_dx37':str, 'i10_dx38':str, 'i10_dx39':str, 'i10_dx40':str, 'elective':'float16', 'female':'float16', 'hcup_ed':'float16', 'hosp_nrd':'float64', 'los': 'float16', 'mdc':'float16', 'mdc_nopoa':'float16', 'i10_ndx':'float16', 'i10_npr':'float16', 'nrd_daystoevent':'float16', 'nrd_stratum':'float16', 'nrd_visitlink':str, 'pay1':'float16', 'pl_nchs':'float16', 'i10_pr1':str, 'i10_pr2':str, 'i10_pr3':str, 'i10_pr4':str, 'i10_pr5':str, 'i10_pr6':str, 'i10_pr7':str, 'i10_pr8':str, 'i10_pr9':str, 'i10_pr10':str, 'i10_pr11':str, 'i10_pr12':str, 'i10_pr13':str, 'i10_pr14':str, 'i10_pr15':str, 'i10_pr16':str, 'i10_pr17':str, 'i10_pr18':str, 'i10_pr19':str, 'i10_pr20':str, 'i10_pr21':str, 'i10_pr22':str, 'i10_pr23':str, 'i10_pr24':str, 'i10_pr25':str, 'prday1':'float16', 'prday2':'float16', 'prday3':'float16', 'prday4':'float16', 'prday5':'float16', 'prday6':'float16', 'prday7':'float16', 'prday8':'float16', 'prday9':'float16', 'prday10':'float16', 'prday11':'float16', 'prday12':'float16', 'prday13':'float16', 'prday14':'float16', 'prday15':'float16', 'prday16':'float16', 'prday17':'float16', 'prday18':'float16', 'prday19':'float16', 'prday20':'float16', 'prday21':'float16', 'prday22':'float16', 'prday23':'float16', 'prday24':'float16', 'prday25':'float16', 'rehabtransfer':'float16', 'resident':'float16', 'samedayevent':'float16', 'totchg':'float64', 'year':'float16', 'zipinc_qrtl':'float16', 'dxver':'float16', 'prver':'float16', 'preventable_readmission':'float16', 'prev_readm_reason':'float16', 'medical_admission':'float16', 'hfrs_score':'float16', 'hfrs_severity':'float16', 'cc_score':'float16', 'cc_severity':'float16'})

In [3]:
# Number of unique patients in the entire file.
len(np.unique(cohort_file['nrd_visitlink']))

62472

In [4]:
# Now we need to carve out the cohort by removing any patients who are younger than 18 and do not have an IBD diagnosis between January - June 2017 (the first 6 months).

# These commented out methods do not work because there are some patients who were 17 initially but turned 18 at a later diagnosis. We are removing those patients as they were not
# adults when first receiving a diagnosis.
# ibd_file = pd.concat([pd.DataFrame([row],columns = row._fields) for row in tqdm(cohort_file.itertuples(index=False)) if row.age > 17]) # slower than using .loc
# ibd_file = cohort_file.loc[cohort_file['age'] > 17]

# First we'll create a list of patients who are not adults (ages 17 and under) and filter it for unique nrd_visitlink ID's.
not_adult = [row.nrd_visitlink for row in tqdm(cohort_file.itertuples(index=False)) if row.age < 18]
not_adult_unique = np.unique(not_adult)

print(f'\n{len(not_adult_unique)}')

# Then we will create a new dataframe (defined_cohort_file) with nrd_visitlink's not in the list, therefore creating a cohort of only adults (ages 18+).
defined_cohort_file = pd.concat([pd.DataFrame([row],columns = row._fields) for row in tqdm(cohort_file.itertuples(index=False)) if row.nrd_visitlink not in not_adult_unique])

118288it [00:01, 59346.95it/s]
0it [00:00, ?it/s]
2572
118288it [07:59, 246.81it/s]


In [5]:
# Number of unique patients in the entire file.
len(np.unique(defined_cohort_file['nrd_visitlink']))

59900

In [6]:
# Create a list of patient ID's who have an IBD related index hospitalization between January - June 2017 (the first 6 months).
patient_list = []
patient_list = [row.nrd_visitlink for row in tqdm(defined_cohort_file.itertuples(index=False)) if row.dmonth < 7 and 'K50' in row.i10_dx1 or row.dmonth < 7 and 'K51' in row.i10_dx1 or row.dmonth < 7 and 'K50' in row.i10_dx2 or row.dmonth < 7 and 'K51' in row.i10_dx2]

114037it [00:01, 102670.06it/s]


In [7]:
# A new list of unique patient ID's from patient_list.
unique_patients = np.unique(patient_list)
len(unique_patients)

31910

In [8]:
# Go through the dataset and use concat to append any rows that have nrd_visitlinks in the unique_patients list to final_cohort_file (a new dataframe) because these are the patients we are looking at.
final_cohort_file = pd.concat([pd.DataFrame([row],columns = row._fields) for row in tqdm(defined_cohort_file.itertuples(index=False)) if row.nrd_visitlink in unique_patients])

114037it [04:59, 381.22it/s]


In [9]:
# Create a new column called 'ibd_diagnosis' and fill every row with a 0 to start.
final_cohort_file['ibd_diagnosis'] = 0

In [10]:
# Reset the index and drop the old index.
final_cohort_file = final_cohort_file.reset_index(drop=True)

In [11]:
# Start with a placeholder for nrd_visitlink that is made up ('aaaaaa'). This will hold the previous rows nrd_visitlink in each iteration.
# Keep a counter for each successive ibd_diagnosis that a patient has, initialized with 1.
# row_num keeps track of which row we are on.
id_placeholder = 'aaaaaa'
counter = 1
row_num = 0

# This loop goes through each row and compares the nrd_visitlink with that of the previous row. 
for row in tqdm(final_cohort_file.itertuples(index=False)):
    id_current = row.nrd_visitlink
    # If the current and previous nrd_visitlink are not the same and the patient has a ibd related discharge, then a 1 is populated in ibd_diagnosis. Counter reset to 1.
    if id_current != id_placeholder and 'K50' in row.i10_dx1 or id_current != id_placeholder and 'K51' in row.i10_dx1 or id_current != id_placeholder and 'K50' in row.i10_dx2 or id_current != id_placeholder and 'K51' in row.i10_dx2:
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = 1
        id_placeholder = id_current
        counter = 1
    # If the current and previous nrd_visitlink are the same and the patient has a idb related discharge, then the counter increments by 1 and the appropriate number is populated in
    # ibd_diagnosis (each subsequent ibd discharge adds 1 to the patients counter).
    elif id_current == id_placeholder and 'K50' in row.i10_dx1 or id_current == id_placeholder and 'K51' in row.i10_dx1 or id_current == id_placeholder and 'K50' in row.i10_dx2 or id_current == id_placeholder and 'K51' in row.i10_dx2:
        counter += 1
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = counter
        id_placeholder = id_current
    # If the current and previous nrd_visitlink are the same and the patient does not have an ibd related discharge, then a 0 is populated in ibd_diagnosis.
    elif id_current == id_placeholder and 'K50' not in row.i10_dx1 or id_current == id_placeholder and 'K51' not in row.i10_dx1 or id_current == id_placeholder and 'K50' not in row.i10_dx2 or id_current == id_placeholder and 'K51' not in row.i10_dx2:
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = 0
        id_placeholder = id_current
    # If the current and previous nrd_visitlink are not the same and the patient does not have an idb related discharge, then a 0 is populated in ibd_diagnosis. Counter reset to 0.
    # id_placeholder is updated in all cases to the current nrd_visitlink.
    else:
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = 0
        id_placeholder = id_current
        counter = 0
    row_num += 1

66060it [00:01, 51734.25it/s]


In [12]:
# Create a new column called 'ibd_index_hospitalization' and fill each row with a 0.
# Use vectorization with masking to find each index hospitalization and fill in a 1 in ibd_index_hospitalization.
final_cohort_file['ibd_index_hospitalization'] = 0
mask = (final_cohort_file['ibd_diagnosis'] == 1)
final_cohort_file.loc[mask, 'ibd_index_hospitalization'] = 1

In [13]:
# Check the file structure.
final_cohort_file

,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year,zipinc_qrtl,dxver,prver,preventable_readmission,prev_readm_reason,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,ibd_diagnosis,ibd_index_hospitalization
0,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,NaN,NaN,1.0,2.000000,0.0,1.0,0.0,1,1
1,63.0,0.0,0.0,1.984195,1.0,3.0,1.0,386.0,34.0,386.0,K50912,E039,M1990,K219,I10,R630,F17210,G43909,Z885,E785,Z79891,Z7982,M549,J449,G8929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,42416.0,2.0,6.0,6.0,15.0,0.0,13000.0,461.0,e0021xf,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,38599.0,2017.0,3.0,10.0,10.0,NaN,NaN,1.0,2.400391,0.0,1.0,0.0,1,1
2,54.0,0.0,0.0,1.389702,1.0,2.0,1.0,389.0,34.0,389.0,K5669,K5010,E663,Z885,Z6826,I10,K5900,F909,F3342,Z883,K5732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,43509.0,3.0,6.0,6.0,11.0,0.0,18352.0,441.0,e002hi0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,28192.0,2017.0,4.0,10.0,10.0,NaN,NaN,1.0,1.799805,0.0,0.0,0.0,1,1
3,55.0,0.0,0.0,1.442027,1.0,11.0,4.0,330.0,35.0,330.0,K5720,K660,F3342,K5010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,42947.0,6.0,6.0,6.0,4.0,2.0,18640.0,496.0,e002hi0,3.0,1.0,0T788DZ,0DBN4ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,103023.0,2017.0,4.0,10.0,10.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0
4,29.0,0.0,0.0,2.085694,1.0,1.0,1.0,386.0,34.0,386.0,K50012,Z87891,Z9049,F329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,43848.0,2.0,6.0,6.0,4.0,0.0,15488.0,406.0,e002vve,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3735.0,2017.

## key_nrd and other variables from hospital and severity files

In [14]:
# Reading in the core file to capture 'key_nrd' as it was dropped on the original merge at the very end. Sort it the same way as final_cohort_file is sorted.
core_file = pd.read_csv('NRD_2017_Core.csv', usecols=['dmonth', 'key_nrd', 'nrd_visitlink'], dtype={'dmonth':'float16', 'key_nrd':'float64', 'nrd_visitlink':str}).sort_values(by=['nrd_visitlink', 'dmonth']).reset_index(drop=True)

# Only keep the patients who are in the list of unique patient nrd_visitlink's from above.
core_file = core_file[core_file['nrd_visitlink'].isin(unique_patients)]
core_file = core_file.reset_index(drop=True)

# Create a dictionary with nrd_visitlink's and their respective key_nrd values and map it to the new 'key_nrd' column in final_cohort_file.
mapping = dict(core_file[['nrd_visitlink', 'key_nrd']].values)
final_cohort_file['key_nrd'] = final_cohort_file['nrd_visitlink'].map(mapping)

print('key_nrd mapping complete')

key_nrd mapping complete


In [15]:
# Reading in the severity and hospital files as well and bringing in variables that were not captured but are part of the variable list.
hospital_2017 = pd.read_csv('NRD_2017_Hospital.csv', usecols=['hosp_bedsize', 'h_contrl', 'hosp_nrd', 'hosp_urcat4', 'hosp_ur_teach', 'n_disc_u', 'n_hosp_u', 's_disc_u', 's_hosp_u', 'total_disc'], dtype={'hosp_bedsize':'float16', 'h_contrl':'float16', 'hosp_nrd':'float64', 'hosp_urcat4':'float16', 'hosp_ur_teach':'float16', 'n_disc_u':'float64', 'n_hosp_u':'float16', 's_disc_u':'float64', 's_hosp_u':'float16', 'total_disc':'float64'})

severity_2017 = pd.read_csv('NRD_2017_Severity.csv', dtype={'aprdrg':'float16', 'aprdrg_risk_mortality':'float16', 'aprdrg_severity':'float16', 'hosp_nrd':'float64', 'key_nrd':'float64'})

final_cohort_file = pd.merge(final_cohort_file, hospital_2017, how='left', on='hosp_nrd')
print('hospital merge complete')

final_cohort_file = pd.merge(final_cohort_file, severity_2017, how='left', on='key_nrd')
final_cohort_file = final_cohort_file.drop(columns=['hosp_nrd_y'])
final_cohort_file = final_cohort_file.rename(columns={'hosp_nrd_x':'hosp_nrd'})
print('severity merge complete')

hospital merge complete
severity merge complete


In [21]:
# Take a look at the file and make sure everything makes sense so far.
final_cohort_file

,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year,zipinc_qrtl,dxver,prver,preventable_readmission,prev_readm_reason,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,ibd_diagnosis,ibd_index_hospitalization,key_nrd,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,aprdrg,aprdrg_risk_mortality,aprdrg_severity,intestinal_abscess,meningitis,encephalitis,endocarditis,pneumonia,pyelonephritis,septic_arthritis_osteomyelitis,septicemia_or_bacteremia,clostridium_difficile,pulmonary_tuberculosis,atypical_mycobacteria,cryptococcosis_aspergillosis_histoplasmosis,listeriosis,leishmaniasis,Pneumocystis_jiroveci_pneumonia,acute_myocardial_infarction,heart_failure,unstable_angina,arrhythmia,cardiac_arrest,vte_diagnosis_pe_or_dvt,history_of_vte,systemic_steroid_use,chronic_pain,opioid_overuse,depression,anxiety_disorder,bipolar_disorder,schizophrenia,problems_related_to_education_and_literacy,problems_related_to_employment_and_unemployment,occupational_exposure_to_risk_factors,problems_related_to_housing_and_economic_circumstances,problems_related_to_social_environment,problems_related_to_upbringing,other_problems_related_to_primary_support_group_including_family_circumstances,problems_related_to_certain_psychosocial_circumstances,problems_related_to_other_psychosocial_circumstances,intestinal_surgery,flexible_sigmoidoscopy_colonoscopy,blood_transfusion,parenteral_or_enteral_nutrition
0,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,NaN,NaN,1.0,2.000000,0.0,1.0,0.0,1,1,2.017057e+11,2.0,2.0,4.0,2.0,18409.0,9.0,11959.0,7.0,2872.0,245.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,63.0,0.0,0.0,1.984195,1.0,3.0,1.0,386.0,34.0,386.0,K50912,E039,M1990,K219,I10,R630,F17210,G43909,Z885,E785,Z79891,Z7982,M549,J449,G8929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,42416.0,2.0,6.0,6.0,15.0,0.0,13000.0,461.0,e0021xf,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,38599.0,2017.0,3.0,10.0,10.0,NaN,NaN,1.0,2.400391,0.0,1.0,0.0,1,1,2.017132e+11,3.0,2.0,3.0,2.0,153598.0,41.0,70994.0,23.0,4374.0,245.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,54.0,0.0,0.0,1.389702,1.0,2.0,1.0,389.0,34.0,389.0,K5669,K5010,E663,Z885,Z6826,I10,K5900,F909,F3342,Z883,K5732,NaN,NaN,Na

## Diagnosis Codes

In [17]:
# See notes for the first line of code in this block. The rest of the code is similar but for different variables (column names)

# -- SERIOUS INFECTIONS -- #

# Intestinal abscess - ICD10: K630
# If there is a discharge code for intestinal abscess (ICD10 code: K630) in columns
# i10_dx1 - i10_dx40 then fill in a 1 in the new 'ibd_related_procedure' column, otherwise fill in a 0.
final_cohort_file['intestinal_abscess'] = final_cohort_file.filter(like='i10_dx').isin(['K630']).any(axis=1).astype(int)

# Meningitis - ICD10: G00, G01, G02, G03, A87
final_cohort_file['meningitis'] = final_cohort_file.filter(like='i10_dx').isin(['G00', 'G01', 'G02', 'G03', 'A87']).any(axis=1).astype(int)

# Encephalitis - ICD10: G040, G042, G043, G0481, G0490
final_cohort_file['encephalitis'] = final_cohort_file.filter(like='i10_dx').isin(['G040', 'G042', 'G043', 'G0481', 'G0490']).any(axis=1).astype(int)

# Endocarditis - ICD10: I33, I38, I39
final_cohort_file['endocarditis'] = final_cohort_file.filter(like='i10_dx').isin(['I33', 'I38', 'I39']).any(axis=1).astype(int)

# Pneumonia - ICD10: J11, J12, J13, J14, J15, J16, J17
final_cohort_file['pneumonia'] = final_cohort_file.filter(like='i10_dx').isin(['J11', 'J12', 'J13', 'J14', 'J15', 'J16', 'J17']).any(axis=1).astype(int)

# Pyelonephritis - ICD10: N10
final_cohort_file['pyelonephritis'] = final_cohort_file.filter(like='i10_dx').isin(['N10']).any(axis=1).astype(int)

# Septic arthritis, Osteomyelitis - ICD10: M00, M01, M86
final_cohort_file['septic_arthritis_osteomyelitis'] = final_cohort_file.filter(like='i10_dx').isin(['M00', 'M01', 'M86']).any(axis=1).astype(int)

# Septicemia or bacteremia - ICD10: A41, R7881
final_cohort_file['septicemia_or_bacteremia'] = final_cohort_file.filter(like='i10_dx').isin(['A41', 'R7881']).any(axis=1).astype(int)

# Clostridium difficile - ICD10: A047
final_cohort_file['clostridium_difficile'] = final_cohort_file.filter(like='i10_dx').isin(['A047']).any(axis=1).astype(int)

# -- OPPORTUNISTIC INFECTIONS -- #

# Pulmonary tuberculosis - ICD10: A15
final_cohort_file['pulmonary_tuberculosis'] = final_cohort_file.filter(like='i10_dx').isin(['A15']).any(axis=1).astype(int)

# Atypical mycobacteria - ICD10: A31
final_cohort_file['atypical_mycobacteria'] = final_cohort_file.filter(like='i10_dx').isin(['A31']).any(axis=1).astype(int)

# Cryptococcosis, Aspergillosis, Histoplasmosis - ICD10: B45, B44, B39
final_cohort_file['cryptococcosis_aspergillosis_histoplasmosis'] = final_cohort_file.filter(like='i10_dx').isin(['B45', 'B44', 'B39']).any(axis=1).astype(int)

# Listeriosis - ICD10: A32
final_cohort_file['listeriosis'] = final_cohort_file.filter(like='i10_dx').isin(['A32']).any(axis=1).astype(int)

# Leishmaniasis - ICD10: B55
final_cohort_file['leishmaniasis'] = final_cohort_file.filter(like='i10_dx').isin(['B55']).any(axis=1).astype(int)

# Pneumocystis jiroveci pneumonia - ICD10: B59
final_cohort_file['Pneumocystis_jiroveci_pneumonia'] = final_cohort_file.filter(like='i10_dx').isin(['B59']).any(axis=1).astype(int)

# -- CARDIAC -- #

# Acute myocardial infarction - ICD10: I21
final_cohort_file['acute_myocardial_infarction'] = final_cohort_file.filter(like='i10_dx').isin(['I21']).any(axis=1).astype(int)

# Heart failure - ICD10: I50
final_cohort_file['heart_failure'] = final_cohort_file.filter(like='i10_dx').isin(['I50']).any(axis=1).astype(int)

# Unstable angina - ICD10: I200
final_cohort_file['unstable_angina'] = final_cohort_file.filter(like='i10_dx').isin(['I200']).any(axis=1).astype(int)

# Arrhythmia - ICD10: I44, I45, I47, I48, I49
final_cohort_file['arrhythmia'] = final_cohort_file.filter(like='i10_dx').isin(['I44', 'I45', 'I47', 'I48', 'I49']).any(axis=1).astype(int)

# Cardiac arrest - ICD10: I46
final_cohort_file['cardiac_arrest'] = final_cohort_file.filter(like='i10_dx').isin(['I46']).any(axis=1).astype(int)

# -- VENOUS THROMBOEMBOLISM -- #

# VTE diagnosis (PE or DVT) - ICD10: I824, I260, I269
final_cohort_file['vte_diagnosis_pe_or_dvt'] = final_cohort_file.filter(like='i10_dx').isin(['I824', 'I260', 'I269']).any(axis=1).astype(int)

# History of VTE - ICD10: Z8671
final_cohort_file['history_of_vte'] = final_cohort_file.filter(like='i10_dx').isin(['Z8671']).any(axis=1).astype(int)

# -- STEROID USE -- #

# Systemic steroid use - ICD10: Z7952
final_cohort_file['systemic_steroid_use'] = final_cohort_file.filter(like='i10_dx').isin(['Z7952']).any(axis=1).astype(int)

# -- CHRONIC PAIN -- #

# Chronic pain - ICD10: G894
final_cohort_file['chronic_pain'] = final_cohort_file.filter(like='i10_dx').isin(['G894']).any(axis=1).astype(int)

# -- OPIOID USE -- #

# Opioid overuse - ICD10: F111, T400
final_cohort_file['opioid_overuse'] = final_cohort_file.filter(like='i10_dx').isin(['F111', 'T400']).any(axis=1).astype(int)

# -- PSYCHIATRIC BURDEN -- #

# Depression - ICD10: F32, F33, F34
final_cohort_file['depression'] = final_cohort_file.filter(like='i10_dx').isin(['F32', 'F33', 'F34']).any(axis=1).astype(int)

# Anxiety disorder - ICD10: F40, F41
final_cohort_file['anxiety_disorder'] = final_cohort_file.filter(like='i10_dx').isin(['F40', 'F41']).any(axis=1).astype(int)

# Bipolar disorder - ICD10: F31
final_cohort_file['bipolar_disorder'] = final_cohort_file.filter(like='i10_dx').isin(['F31']).any(axis=1).astype(int)

# Schizophrenia - ICD10: F32, F33, F34
final_cohort_file['schizophrenia'] = final_cohort_file.filter(like='i10_dx').isin(['F20', 'F25']).any(axis=1).astype(int)

# -- SOCIAL DETERMINANTS OF HEALTH -- #

# Problems related to education and literacy - ICD10: Z55
final_cohort_file['problems_related_to_education_and_literacy'] = final_cohort_file.filter(like='i10_dx').isin(['Z55']).any(axis=1).astype(int)

# Problems related to employment and unemployment - ICD10: Z56
final_cohort_file['problems_related_to_employment_and_unemployment'] = final_cohort_file.filter(like='i10_dx').isin(['Z56']).any(axis=1).astype(int)

# Occupational exposure to risk factors - ICD10: Z57
final_cohort_file['occupational_exposure_to_risk_factors'] = final_cohort_file.filter(like='i10_dx').isin(['Z57']).any(axis=1).astype(int)

# Problems related to housing and economic circumstances - ICD10: Z59
final_cohort_file['problems_related_to_housing_and_economic_circumstances'] = final_cohort_file.filter(like='i10_dx').isin(['Z59']).any(axis=1).astype(int)

# Problems related to social environment - ICD10: Z60
final_cohort_file['problems_related_to_social_environment'] = final_cohort_file.filter(like='i10_dx').isin(['Z60']).any(axis=1).astype(int)

# Problems related to upbringing - ICD10: Z62
final_cohort_file['problems_related_to_upbringing'] = final_cohort_file.filter(like='i10_dx').isin(['Z62']).any(axis=1).astype(int)

# Other problems related to primary support group, including family circumstances - ICD10: Z63
final_cohort_file['other_problems_related_to_primary_support_group_including_family_circumstances'] = final_cohort_file.filter(like='i10_dx').isin(['Z63']).any(axis=1).astype(int)

# Problems related to certain psychosocial circumstances - ICD10: Z64
final_cohort_file['problems_related_to_certain_psychosocial_circumstances'] = final_cohort_file.filter(like='i10_dx').isin(['Z64']).any(axis=1).astype(int)

# Problems related to other psychosocial circumstances - ICD10: Z65
final_cohort_file['problems_related_to_other_psychosocial_circumstances'] = final_cohort_file.filter(like='i10_dx').isin(['Z65']).any(axis=1).astype(int)

## Procedure Codes

In [18]:
# -- SURGERY -- #

# Intestinal surgery - ICD10: 0DB80ZZ, 0DB83ZZ, 0DB84ZZ, 0DB87ZZ, 0DB88ZZ, 0DT90ZZ, 0DT94ZZ , 0DT97ZZ , 0DT98ZZ, 0DTA0ZZ, 0DTA4ZZ, 0DTA7ZZ, 0DTA8ZZ, 0DTB0ZZ, 0DTB4ZZ, 0DTB7ZZ, 0DTB8ZZ, 0DT80ZZ, 0DT84ZZ, 0DT87ZZ, 0DT88ZZ, 0DBE0ZZ, 0DBE3ZZ, 0DBE7ZZ, 0DBE8ZZ, 0DTH0ZZ, 0DTH7ZZ, 0DTH8ZZ, 0DTF0ZZ, 0DTF7ZZ, 0DTF8ZZ, 0DTK0ZZ, 0DTL0ZZ, 0DTL7ZZ, 0DTL8ZZ, 0DTG0ZZ, 0DTG7ZZ, 0DTG8ZZ, 0DTN0ZZ, 0DTN7ZZ, 0DTN8ZZ, 0DBE0ZZ, 0DBE3ZZ, 0DBE7ZZ, 0DBE8ZZ, 0DTE4ZZ, 0DTE0ZZ, 0DTE7ZZ, 0DTE8ZZ, 0DTP0ZZ, 0DTP4ZZ, 0DTP7ZZ, 0DTP8ZZ, 0DTP4ZZ, 0DTP0ZZ, 0DTP0ZZ, 0DTP4ZZ, 0D1N0Z4, 0D1N4Z4, 0DBP0ZZ, 0DBP4ZZ, 0DQ80ZZ, 0DQ83ZZ, 0DQ84ZZ, 0DQ87ZZ, 0DQ88ZZ, 0DQA0ZZ, 0DQA3ZZ, 0DQA4ZZ, 0DQA7ZZ, 0DQA8ZZ, 0DQE0ZZ, 0DQE3ZZ, 0DQE4ZZ, 0DQE7ZZ, 0DQE8ZZ, 0DQN0ZZ, 0DQN3ZZ, 0DQN4ZZ, 0DQN7ZZ, 0DQN8ZZ, 0DQP0ZZ, 0DQP3ZZ, 0DQP4ZZ, 0DQP7ZZ, 0DQP8ZZ, 0DQQ0ZZ, 0DQQ3ZZ, 0DQQ4ZZ, 0DQQ7ZZ, 0DQQ8ZZ, 0DQQXZZ, 0H89XZZ, 0DBQ0ZZ, 0DBQ3ZZ, 0DBQ4ZZ, 0TQB0ZZ, 0TQB3ZZ, 0TQB4ZZ, 0TQB7ZZ, 0TQB8ZZ, 0UQG0ZZ, 0UQG3ZZ, 0UQG4ZZ, 0UQG7ZZ, 0UQG8ZZ
final_cohort_file['intestinal_surgery'] = final_cohort_file.filter(like='i10_pr').isin(['0DB80ZZ', '0DB83ZZ', '0DB84ZZ', '0DB87ZZ', '0DB88ZZ', '0DT90ZZ', '0DT94ZZ' , '0DT97ZZ' , '0DT98ZZ', '0DTA0ZZ', '0DTA4ZZ', '0DTA7ZZ', '0DTA8ZZ', '0DTB0ZZ', '0DTB4ZZ', '0DTB7ZZ', '0DTB8ZZ', '0DT80ZZ', '0DT84ZZ', '0DT87ZZ', '0DT88ZZ', '0DBE0ZZ', '0DBE3ZZ', '0DBE7ZZ', '0DBE8ZZ', '0DTH0ZZ', '0DTH7ZZ', '0DTH8ZZ', '0DTF0ZZ', '0DTF7ZZ', '0DTF8ZZ', '0DTK0ZZ', '0DTL0ZZ', '0DTL7ZZ', '0DTL8ZZ', '0DTG0ZZ', '0DTG7ZZ', '0DTG8ZZ', '0DTN0ZZ', '0DTN7ZZ', '0DTN8ZZ', '0DBE0ZZ', '0DBE3ZZ', '0DBE7ZZ', '0DBE8ZZ', '0DTE4ZZ', '0DTE0ZZ', '0DTE7ZZ', '0DTE8ZZ', '0DTP0ZZ', '0DTP4ZZ', '0DTP7ZZ', '0DTP8ZZ', '0DTP4ZZ', '0DTP0ZZ', '0DTP0ZZ', '0DTP4ZZ', '0D1N0Z4', '0D1N4Z4', '0DBP0ZZ', '0DBP4ZZ', '0DQ80ZZ', '0DQ83ZZ', '0DQ84ZZ', '0DQ87ZZ', '0DQ88ZZ', '0DQA0ZZ', '0DQA3ZZ', '0DQA4ZZ', '0DQA7ZZ', '0DQA8ZZ', '0DQE0ZZ', '0DQE3ZZ', '0DQE4ZZ', '0DQE7ZZ', '0DQE8ZZ', '0DQN0ZZ', '0DQN3ZZ', '0DQN4ZZ', '0DQN7ZZ', '0DQN8ZZ', '0DQP0ZZ', '0DQP3ZZ', '0DQP4ZZ', '0DQP7ZZ', '0DQP8ZZ', '0DQQ0ZZ', '0DQQ3ZZ', '0DQQ4ZZ', '0DQQ7ZZ', '0DQQ8ZZ', '0DQQXZZ', '0H89XZZ', '0DBQ0ZZ', '0DBQ3ZZ', '0DBQ4ZZ', '0TQB0ZZ', '0TQB3ZZ', '0TQB4ZZ', '0TQB7ZZ', '0TQB8ZZ', '0UQG0ZZ', '0UQG3ZZ', '0UQG4ZZ', '0UQG7ZZ', '0UQG8ZZ']).any(axis=1).astype(int)

# -- PROCEDURES -- #

# Flexible sigmoidoscopy/colonoscopy - ICD10: 0DJD8ZZ, 0D9E8ZX, 0D9H8ZX, 0D9N8ZX, 0DBE8ZX, 0DBH8ZX, 0DBN8ZX, 0DBE8ZZ
final_cohort_file['flexible_sigmoidoscopy_colonoscopy'] = final_cohort_file.filter(like='i10_pr').isin(['0DJD8ZZ', '0D9E8ZX', '0D9H8ZX', '0D9N8ZX', '0DBE8ZX', '0DBH8ZX', '0DBN8ZX', '0DBE8ZZ']).any(axis=1).astype(int)

# Blood Transfusion - ICD10: 30233H1, 30243H1, 30253H1, 30263H1, 30233H0, 30233N0, 30233W0, 30243H0, 30243N0, 30243W0, 30253H0, 30253N0, 30253W0,  30263H0, 30263N0, 30263W0,   30233N1, 30233P1, 30243N1, 30243P1, 30253N1, 30253P1, 30263N1, 30263P1, 30233R1, 30243R1, 30253R1, 30263R1
final_cohort_file['blood_transfusion'] = final_cohort_file.filter(like='i10_pr').isin(['30233H1', '30243H1', '30253H1', '30263H1', '30233H0', '30233N0', '30233W0', '30243H0', '30243N0', '30243W0', '30253H0', '30253N0', '30253W0', '30263H0', '30263N0', '30263W0', '30233N1', '30233P1', '30243N1', '30243P1', '30253N1', '30253P1', '30263N1', '30263P1', '30233R1', '30243R1', '30253R1', '30263R1']).any(axis=1).astype(int)

# Parenteral or enteral nutrition - ICD10: 0DH67UZ, 0DH68UZ, 3E0G36Z, 3E0336Z, 3E0436Z, 3E0536Z, 3E0636Z
final_cohort_file['parenteral_or_enteral_nutrition'] = final_cohort_file.filter(like='i10_pr').isin(['0DH67UZ', '0DH68UZ', '3E0G36Z', '3E0336Z', '3E0436Z', '3E0536Z', '3E0636Z']).any(axis=1).astype(int)

In [22]:
# Save the file as a CSV.
final_cohort_file.to_csv("NRD_2017_Defined_Cohort.csv", index=False)

## Calculated fields

In [15]:
# If there is an IBD related procedure (flexible sigmoidoscopy or colonoscopy, ICD10 codes: 0DJD8ZZ, 0D9.E8ZX, 0D9.H8ZX, 0D9.N8ZX, 0DB.E8ZX, 0DB.H8ZX, 0DB.N8ZX, 0DB.E8ZZ) in columns
# i10_pr1 - i10_pr25 then fill in a 1 in the new 'ibd_related_procedure' column, otherwise fill in a 0.
final_cohort_file['ibd_related_procedures'] = final_cohort_file.filter(like='i10_pr').isin(['0DJD8ZZ', '0D9E8ZX', '0D9H8ZX', '0D9N8ZX', '0DBE8ZX', '0DBH8ZX', '0DBN8ZX', '0DBE8ZZ']).any(axis=1).astype(int)

In [17]:
# Use pivot_table with aggregate function 'sum' to essentially group by the nrd_visitlinks and sum the ibd_related_procedures for each patient.
pivot_df = final_cohort_file.pivot_table(index='nrd_visitlink', values='ibd_related_procedures', aggfunc='sum').reset_index()

In [18]:
# Print out the number of times each value shows up. Value is the total number of ibd_related_procedures and counts are the number of patients who had that many procedures.
values, counts = np.unique(pivot_df['ibd_related_procedures'], return_counts=True)
print('Values:  Counts:')
for number in values:
    print(f'{values[number]}        {counts[number]}')

Values:  Counts:
0        25407
1        5944
2        477
3        68
4        13
5        1


In [19]:
# Since the pivot table only has unique patient nrd_visitlink's and totals per patient, create a dictionary to store each patient and their total ibd_related_procedures.
patient_totals = {}
for row in pivot_df.itertuples(index=False):
    patient_totals.update({row.nrd_visitlink: row.ibd_related_procedures})

In [20]:
# Double check to make sure we still have the same number of unique patients as before.
len(patient_totals)

31910

In [21]:
# Create a new column called 'total_ibd_related_procedures' by mapping the nrd_visitlinks values from the dicionary patient_totals.
final_cohort_file['total_ibd_related_procedures'] = final_cohort_file['nrd_visitlink'].map(patient_totals)

In [22]:
# Take a quick look at the file.
final_cohort_file

,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year,zipinc_qrtl,dxver,prver,preventable_readmission,prev_readm_reason,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,ibd_diagnosis,ibd_index_hospitalization,ibd_related_procedures,total_ibd_related_procedures
0,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,NaN,NaN,1.0,2.000000,0.0,1.0,0.0,1,1,0,0
1,63.0,0.0,0.0,1.984195,1.0,3.0,1.0,386.0,34.0,386.0,K50912,E039,M1990,K219,I10,R630,F17210,G43909,Z885,E785,Z79891,Z7982,M549,J449,G8929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,42416.0,2.0,6.0,6.0,15.0,0.0,13000.0,461.0,e0021xf,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,38599.0,2017.0,3.0,10.0,10.0,NaN,NaN,1.0,2.400391,0.0,1.0,0.0,1,1,0,0
2,54.0,0.0,0.0,1.389702,1.0,2.0,1.0,389.0,34.0,389.0,K5669,K5010,E663,Z885,Z6826,I10,K5900,F909,F3342,Z883,K5732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,43509.0,3.0,6.0,6.0,11.0,0.0,18352.0,441.0,e002hi0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,28192.0,2017.0,4.0,10.0,10.0,NaN,NaN,1.0,1.799805,0.0,0.0,0.0,1,1,0,0
3,55.0,0.0,0.0,1.442027,1.0,11.0,4.0,330.0,35.0,330.0,K5720,K660,F3342,K5010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,42947.0,6.0,6.0,6.0,4.0,2.0,18640.0,496.0,e002hi0,3.0,1.0,0T788DZ,0DBN4ZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,103023.0,2017.0,4.0,10.0,10.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0
4,29.0,0.0,0.0,2.085694,1.0,1.0,1.0,386.0,34.0,386.0,K50012,Z87891,Z9049,F329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,43848.0,2.0,6.0,6.0,4.0,0.0,15488.0,406.0,e002vve,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [23]:
# Print out the number of times each value shows up.
values, counts = np.unique(final_cohort_file['total_ibd_related_procedures'], return_counts=True)
print('Values:  Counts:')
for number in values:
    print(f'{values[number]}        {counts[number]}')

Values:  Counts:
0        49325
1        13918
2        2210
3        490
4        109
5        8
